In [10]:
import pandas as pd
import numpy as np

RAW_DATA_PATH = '../data/raw/data.csv'
INTERIM_DATA_PATH = '../data/interim/parsed_data.pkl'

data  = pd.read_csv(RAW_DATA_PATH)
data.head()

,Brand,Year,Model,Car/Suv,Title,UsedOrNew,Transmission,Engine,DriveType,FuelType,FuelConsumption,Kilometres,ColourExtInt,Location,CylindersinEngine,BodyType,Doors,Seats,Price
0,Ssangyong,2022.0,Rexton,Sutherland Isuzu Ute,2022 Ssangyong Rexton Ultimate (awd),DEMO,Automatic,"4 cyl, 2.2 L",AWD,Diesel,8.7 L / 100 km,5595,White / Black,"Caringbah, NSW",4 cyl,SUV,4 Doors,7 Seats,51990
1,MG,2022.0,MG3,Hatchback,2022 MG MG3 Auto Excite (with Navigation),USED,Automatic,"4 cyl, 1.5 L",Front,Premium,6.7 L / 100 km,16,Black / Black,"Brookvale, NSW",4 cyl,Hatchback,5 Doors,5 Seats,19990
2,BMW,2022.0,430I,Coupe,2022 BMW 430I M Sport,USED,Automatic,"4 cyl, 2 L",Rear,Premium,6.6 L / 100 km,8472,Grey / White,"Sylvania, NSW",4 cyl,Coupe,2 Doors,4 Seats,108988
3,Mercedes-Benz,2011.0,E500,Coupe,2011 Mercedes-Benz E500 Elegance,USED,Automatic,"8 cyl, 5.5 L",Rear,Premium,11 L / 100 km,136517,White / Brown,"Mount Druitt, NSW",8 cyl,Coupe,2 Doors,4 Seats,32990
4,Renault,2022.0,Arkana,SUV,2022 Renault Arkana Intens,USED,Automatic,"4 cyl, 1.3 L",Front,Unleaded,6 L / 100 km,1035,Grey / Black,"Castle Hill, NSW",4 cyl,SUV,4 Doors,5 Seats,34990


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16734 entries, 0 to 16733
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Brand              16733 non-null  object 
 1   Year               16733 non-null  float64
 2   Model              16733 non-null  object 
 3   Car/Suv            16706 non-null  object 
 4   Title              16733 non-null  object 
 5   UsedOrNew          16733 non-null  object 
 6   Transmission       16733 non-null  object 
 7   Engine             16733 non-null  object 
 8   DriveType          16733 non-null  object 
 9   FuelType           16733 non-null  object 
 10  FuelConsumption    16733 non-null  object 
 11  Kilometres         16733 non-null  object 
 12  ColourExtInt       16733 non-null  object 
 13  Location           16284 non-null  object 
 14  CylindersinEngine  16733 non-null  object 
 15  BodyType           16452 non-null  object 
 16  Doors              151

In [12]:
import re

def extract_int_color(x):
    if pd.isna(x):
        return np.nan
    parts = str(x).split('/')
    return parts[0].strip() if parts else np.nan

def extract_ext_color(x):
    if pd.isna(x):
        return np.nan
    parts = str(x).split('/')
    return parts[1].strip() if parts else np.nan

def extract_first_number(x):
    if pd.isna(x) or str(x).strip() == '-':
        return np.nan
    match = re.search(r'(\d+\.?\d*)', str(x))
    return float(match.group(1)) if match else np.nan

def extract_engine(x):
    if pd.isna(x):
        return np.nan
    numbers = re.findall(r'\d+(?:\.\d+)?', str(x))
    if len(numbers) < 2:
        return np.nan 
    return float(numbers[1])

def extract_location(x):
    if pd.isna(x):
        return np.nan
    parts = str(x).split(',')
    return parts[0].strip() if parts else np.nan

def extract_state(x):
    if pd.isna(x):
        return np.nan
    parts = str(x).split(',')
    return parts[1].strip() if parts else np.nan

data['Doors'] = data['Doors'].apply(extract_first_number)
data['Seats'] = data['Seats'].apply(extract_first_number)
data['CylindersinEngine'] = data['CylindersinEngine'].apply(extract_first_number)
data['FuelConsumption'] = data['FuelConsumption'].apply(extract_first_number)

data['City'] = data['Location'].apply(extract_location)
data['State'] = data['Location'].apply(extract_state)

data['Engine'] = data['Engine'].apply(extract_engine)

data['Colour_Ext'] = data['ColourExtInt'].apply(extract_ext_color)
data['Colour_Int'] = data['ColourExtInt'].apply(extract_int_color)

data.head(2)

,Brand,Year,Model,Car/Suv,Title,UsedOrNew,Transmission,Engine,DriveType,FuelType,...,Location,CylindersinEngine,BodyType,Doors,Seats,Price,City,State,Colour_Ext,Colour_Int
0,Ssangyong,2022.0,Rexton,Sutherland Isuzu Ute,2022 Ssangyong Rexton Ultimate (awd),DEMO,Automatic,2.2,AWD,Diesel,...,"Caringbah, NSW",4.0,SUV,4.0,7.0,51990,Caringbah,NSW,Black,White
1,MG,2022.0,MG3,Hatchback,2022 MG MG3 Auto Excite (with Navigation),USED,Automatic,1.5,Front,Premium,...,"Brookvale, NSW",4.0,Hatchback,5.0,5.0,19990,Brookvale,NSW,Black,Black


In [13]:
categorical_cols = ['Brand', 'Model', 'Car/Suv', 'Title', 'UsedOrNew', 
                    'Transmission', 'DriveType', 'FuelType', 'BodyType', 
                    'City', 'State', 'Colour_Ext', 'Colour_Int']

for col in categorical_cols:
    if col in data.columns:
        data[col] = data[col].astype(str).str.strip().str.title()
        data[col] = data[col].replace({'Nan': np.nan, 'Na': np.nan, 'N/a': np.nan,
                                        'Mq': np.nan, '-': np.nan})

data.head(5)

,Brand,Year,Model,Car/Suv,Title,UsedOrNew,Transmission,Engine,DriveType,FuelType,...,Location,CylindersinEngine,BodyType,Doors,Seats,Price,City,State,Colour_Ext,Colour_Int
0,Ssangyong,2022.0,Rexton,Sutherland Isuzu Ute,2022 Ssangyong Rexton Ultimate (Awd),Demo,Automatic,2.2,Awd,Diesel,...,"Caringbah, NSW",4.0,Suv,4.0,7.0,51990,Caringbah,Nsw,Black,White
1,Mg,2022.0,Mg3,Hatchback,2022 Mg Mg3 Auto Excite (With Navigation),Used,Automatic,1.5,Front,Premium,...,"Brookvale, NSW",4.0,Hatchback,5.0,5.0,19990,Brookvale,Nsw,Black,Black
2,Bmw,2022.0,430I,Coupe,2022 Bmw 430I M Sport,Used,Automatic,2.0,Rear,Premium,...,"Sylvania, NSW",4.0,Coupe,2.0,4.0,108988,Sylvania,Nsw,White,Grey
3,Mercedes-Benz,2011.0,E500,Coupe,2011 Mercedes-Benz E500 Elegance,Used,Automatic,5.5,Rear,Premium,...,"Mount Druitt, NSW",8.0,Coupe,2.0,4.0,32990,Mount Druitt,Nsw,Brown,White
4,Renault,2022.0,Arkana,Suv,2022 Renault Arkana Intens,Used,Automatic,1.3,Front,Unleaded,...,"Castle Hill, NSW",4.0,Suv,4.0,5.0,34990,Castle Hill,Nsw,Black,Grey


In [14]:
data.columns

Index(['Brand', 'Year', 'Model', 'Car/Suv', 'Title', 'UsedOrNew',
       'Transmission', 'Engine', 'DriveType', 'FuelType', 'FuelConsumption',
       'Kilometres', 'ColourExtInt', 'Location', 'CylindersinEngine',
       'BodyType', 'Doors', 'Seats', 'Price', 'City', 'State', 'Colour_Ext',
       'Colour_Int'],
      dtype='object')

In [15]:
final_columns = ['brand', 'year', 'model', 'usedornew',
       'transmission', 'engine', 'drivetype', 'fueltype', 'fuelconsumption', 
       'kilometres', 'cylindersinengine', 'bodytype', 'doors', 'seats', 'price', 
       'city', 'state', 'colour_ext', 'colour_int']

data.columns = data.columns.str.lower()
data['price'] = pd.to_numeric(data['price'], errors='coerce')
data['kilometres'] = pd.to_numeric(data['kilometres'].astype(str).str.replace(',', ''), errors='coerce')
data['year'] = data['year'].astype('Int64')
cols_to_save = [col for col in final_columns if col in data.columns]
data_interim = data[cols_to_save].copy()

In [16]:
data_interim.to_pickle(INTERIM_DATA_PATH)
data_interim.head()

,brand,year,model,usedornew,transmission,engine,drivetype,fueltype,fuelconsumption,kilometres,cylindersinengine,bodytype,doors,seats,price,city,state,colour_ext,colour_int
0,Ssangyong,2022,Rexton,Demo,Automatic,2.2,Awd,Diesel,8.7,5595.0,4.0,Suv,4.0,7.0,51990.0,Caringbah,Nsw,Black,White
1,Mg,2022,Mg3,Used,Automatic,1.5,Front,Premium,6.7,16.0,4.0,Hatchback,5.0,5.0,19990.0,Brookvale,Nsw,Black,Black
2,Bmw,2022,430I,Used,Automatic,2.0,Rear,Premium,6.6,8472.0,4.0,Coupe,2.0,4.0,108988.0,Sylvania,Nsw,White,Grey
3,Mercedes-Benz,2011,E500,Used,Automatic,5.5,Rear,Premium,11.0,136517.0,8.0,Coupe,2.0,4.0,32990.0,Mount Druitt,Nsw,Brown,White
4,Renault,2022,Arkana,Used,Automatic,1.3,Front,Unleaded,6.0,1035.0,4.0,Suv,4.0,5.0,34990.0,Castle Hill,Nsw,Black,Grey
